# Matrix Multiplication with PyCUDA

This notebook demonstrates how to perform matrix multiplication using PyCUDA. The CUDA kernel implementation is left empty for you to complete as an exercise.

In [ ]:
# Import Required Libraries
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

from cuda_helpers import profile_gpu

## Define Matrices

Let's define two matrices to multiply. You can change their size and values as needed.

In [ ]:
# Define two matrices A and B
N = 1024
A = np.random.randn(N, N).astype(np.float32)
B = np.random.randn(N, N).astype(np.float32)
print("Matrix A:\n", A)
print("Matrix B:\n", B)

## Perform Matrix Multiplication on the GPU

We will multiply matrices A and B using a CUDA kernel. Implement the kernel code to perform standard matrix multiplication as done i algebra. For two matrices A (of size m×n) and B (of size n×p), their product C = AB is an m×p matrix where each element C[i, j] is computed as the sum of products of the i-th row of A and the j-th column of B:

$$
C_{i,j} = \sum_{k=1}^{N} A_{i,k} \cdot B_{k,j}
$$

This operation is also known as "GEMM" (General Matrix Multiply) in numerical computing libraries.

We will build-up the optimal solution in stages. Start simple, and implement the mutliplication using just global memory, where each thread computes output value for one cell in the output matrix.

You can assume that all matrices are square for now, if you find it more convenient.

In [ ]:
# Allocate GPU memory and transfer matrices
d_a = cuda.mem_alloc(A.nbytes)
d_b = cuda.mem_alloc(B.nbytes)
d_c = cuda.mem_alloc(A.nbytes)
cuda.memcpy_htod(d_a, A)
cuda.memcpy_htod(d_b, B)

# CUDA kernel for matrix multiplication (to be completed)
kernel_code = '''
__global__ void matmul(float *A, float *B, float *C, int N) {
    // TODO: Implement matrix multiplication kernel
    // blockIdx, blockDim, threadIdx, gridDim
    float sum = 0;
    int2 global_id = make_int2(blockIdx.x * blockDim.x + threadIdx.x,
                               blockIdx.y * blockDim.y + threadIdx.y);

    if (global_id.x >= N || global_id.y >= N) {
        return;
    }
    
    // your code goes here
}
'''

mod = SourceModule(kernel_code)
matmul = mod.get_function("matmul")

In [ ]:
block_size = (8, 8, 1)
grid_size = (A.shape[0] // block_size[0], A.shape[1] // block_size[1], 1)

print(f'Launching with grid_size={grid_size}, block_size={block_size}')

n_warmup = 2
n_iters = 50

launch = lambda: matmul(d_a, d_b, d_c, np.int32(N), block=block_size, grid=grid_size)
_ = profile_gpu(launch, n_warmup=n_warmup, n_iters=n_iters)

## Display Results

After running the kernel, copy the result back to the host and display it.

Refer to the [solution](matrix_multiplication_solution_global.cu) if you get stuck.

In [ ]:
# Copy result from GPU and display
C = np.empty_like(A)
cuda.memcpy_dtoh(C, d_c)
c_numpy = np.matmul(A, B)
print("Result matrix C (A x B):\n", C)

np.testing.assert_almost_equal(C, c_numpy, decimal=3)
# Note: You need to implement the kernel for correct results!

# Shared memory caching

Your next task is to optimize the kernel execution time. One of the common practices in matrix multipication is to optimize memory accesses. Elements in each input matrices A and B are accessed N times by reaching to global memory. While we can expect some level of caching happening under the hood, we can also cache parts of A and B matrices in Shared Local Memory.

Below is a helper function that will launch your new kernel. It will:
* validate the sized of your matrices
* automatically allocate dynamic share memory for matrices A and B
* transfer matrices from host to device and back
* launch your kernel
* verify correctness of the calculations against numpy implementation
* if the results are correct it will launch the kernel multiple times to measure execution times

In [ ]:
def gpu_matrix_multiply(A, B, kernel_code, block_size=(16, 16, 1), warmup=2, iters=100, *args, **kwargs):
    import numpy as np
    import pycuda.driver as cuda
    from pycuda.compiler import SourceModule
    from cuda_helpers import profile_gpu

    M, N_A = A.shape
    N_B, P = B.shape
    print(f"A: MxN ({M}, {N_A}), B: NxP ({N_B}, {P}), C: MxP ({M}, {P})")
    assert N_A == N_B, "Inner matrix dimensions must match"

    d_a = cuda.mem_alloc(A.nbytes)
    d_b = cuda.mem_alloc(B.nbytes)
    d_c = cuda.mem_alloc(M * P * 4)
    cuda.memcpy_htod(d_a, A)
    cuda.memcpy_htod(d_b, B)

    mod = SourceModule(kernel_code)
    matmul = mod.get_function("matmul")

    bx, by, bz = block_size
    grid_x = (P + bx - 1) // bx
    grid_y = (M + by - 1) // by
    grid_size = (grid_x, grid_y, 1)

    use_slm = 'extern __shared__' in kernel_code
    if use_slm:
        nr_threads = bx * by
        shared_mem_bytes = nr_threads * 4 * 2
    else:
        shared_mem_bytes = 0

    def launch():
        if shared_mem_bytes:
            matmul(d_a, d_b, d_c, *args, block=block_size, grid=grid_size, shared=shared_mem_bytes, **kwargs)
        else:
            matmul(d_a, d_b, d_c, *args, block=block_size, grid=grid_size, **kwargs)
            
    print(f'Launching with grid_size={grid_size}, block_size={block_size}, shared_mem_bytes={shared_mem_bytes}')
    launch()

    C = np.empty((M, P), dtype=np.float32)
    cuda.memcpy_dtoh(C, d_c)
    ref = np.matmul(A, B)
    np.testing.assert_almost_equal(C, ref, decimal=3)

    _ = profile_gpu(launch, n_warmup=warmup, n_iters=iters)


## Optimization task

Write an optimized kernel which will cache blocks from A anb B matrices in SLM:
- for similicity assume that the shapes of all matrices are multiple of block size
- make sure there are not data races - so synchronize SLM accesses

Implement the following algorithm:
- for each cache block
    - load 8x8 blocks from global memory into declared SLMs
        - the main difficulty lays in accessing global memory based while iterating over cached blocks
    - produce partial matrix multipication sum from data cached in SLM and accumulate in local variable
        - here you just need local ids
- dump accumulated sum into global memory C

In [ ]:
shared_mem_kernel = '''
    #define THREAD_INDEX (threadIdx.y * blockDim.x + threadIdx.x)

    __global__ void matmul(float *A, float *B, float *C, int N) {
        float sum = 0;
        int2 global_id = make_int2(blockIdx.x * blockDim.x + threadIdx.x,
                                   blockIdx.y * blockDim.y + threadIdx.y);
                
        extern __shared__ float slm[];
        float *slm_A = &slm[0];
        float *slm_B = &slm[blockDim.x * blockDim.y];
        
        // your code goes here
    }
'''

# For square matrices, pass N as kernel argument
args = (np.int32(N),)
gpu_matrix_multiply(A, B, 
                    shared_mem_kernel,
                    (16, 16, 1),
                    2, 100,
                    *args)

Refer to the [solution](matrix_multiplication_solution_slm.cu) if you get stuck.

# Non-square matrices

Adapt your solution to work with non-square matrices. You will to:
* pass sizes of matrices to your kernel - as they are non-square so can have different shapes than only N.
* Use this shapes in your kernel to access elements.
* think about iterating through consecutive cached blocks in the for-loop. Make sure to look iterate from the perspective of output matrix.

In [ ]:
non_square_kernel = '''
    #define THREAD_INDEX (threadIdx.y * blockDim.x + threadIdx.x)

    __global__ void matmul(float *A, float *B, float *C, int M, int N, int P) {
        float sum = 0;
        int2 global_id = make_int2(blockIdx.x * blockDim.x + threadIdx.x,
                                   blockIdx.y * blockDim.y + threadIdx.y);

        extern __shared__ float slm[];
        float *slm_A = &slm[0];
        float *slm_B = &slm[blockDim.x * blockDim.y];
        
        int nr_blocks = (N + blockDim.x - 1) / blockDim.x;

        // your code goes here
    }
'''

A_ = np.random.randn(512, 1024).astype(np.float32)
B_ = np.random.randn(1024, 768).astype(np.float32)
# For non-square matrices, pass M, N, P as kernel arguments
M, N, P = A_.shape[0], A_.shape[1], B_.shape[1]
args = (np.int32(M), np.int32(N), np.int32(P))
gpu_matrix_multiply(A_, B_, 
                    non_square_kernel,
                    (16, 16, 1),
                    2, 100,
                    *args)

Refer to the [solution](matrix_multiplication_solution_slm_non_square.cu) if you get stuck.